# NTB Mall Recommendation System using CBF

## Preprocessing Dataset

### Import library

In [2]:
import pandas as pd
import numpy as np
import math
import nltk
nltk.download("book")
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/mizanul/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package depend

### Load Dataset

In [ ]:
df_base = pd.read_csv("./Clean_Dataset.csv")
df_base = pd.DataFrame(df_base)
df_base

,ID Produk,Tanggal Input,Nama Toko,Email Pelapak,Kategori Produk,Sub Kategori Produk,Nama Produk,Variasi,Harga,Stok,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,2024-04-13 04:24:29,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Permen,Supermen,40 Gr,12000,0,<p>Permen susu khas lombok nusa tenggara barat...
1,066a5e68-7799-4f18-a49a-e2fefc556d1f,2024-04-13 04:24:30,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Permen,Supermen,60 Gr,16000,0,<p>Permen susu khas lombok nusa tenggara barat...
2,b70def04-da59-4c2e-912a-e5944579831d,2023-10-29 07:45:51,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),NaN,15000,37,<p>Coklat susu adalah mahakarya gurih yang mem...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,2023-10-19 09:45:43,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lanang,NaN,19000,20,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...
4,9271605a-009d-4364-8deb-f8d04847e38b,2023-10-19 09:35:33,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lombok Murni,NaN,28000,20,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...
...,...,...,...,...,...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,2021-03-25 13:42:44,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Madu,Mbojo Honey,NaN,169000,0,<p>Mbojo Honey<br></p>
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,2021-03-25 13:38:11,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,NaN,29000,0,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,2021-03-25 13:34:01,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,NaN,29000,0,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,2021-03-25 13:32:03,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,NaN,455000,5,<p>Dompet Motif Krealang Sumbawa<br></p>


In [ ]:
df_preprocess  = df_base.drop(["Tanggal Input", "Nama Toko", "Email Pelapak", "Kategori Produk", "Variasi", "Stok"], axis=1)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1916 non-null   object
 1   Sub Kategori Produk  1916 non-null   object
 2   Nama Produk          1916 non-null   object
 3   Harga                1916 non-null   int64 
 4   Deksripsi            1916 non-null   object
dtypes: int64(1), object(4)
memory usage: 75.0+ KB


### Check Missing Values and Data Duplications

In [ ]:
df_preprocess.isnull().sum()

ID Produk              0
Sub Kategori Produk    0
Nama Produk            0
Harga                  0
Deksripsi              0
dtype: int64

In [ ]:
df_preprocess['Sub Kategori Produk'] = df_preprocess['Sub Kategori Produk'].astype(str)
df_preprocess['Nama Produk'] = df_preprocess['Nama Produk'].astype(str)
df_preprocess['Harga'] = df_preprocess['Harga'].astype(str)
df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].astype(str)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1916 non-null   object
 1   Sub Kategori Produk  1916 non-null   object
 2   Nama Produk          1916 non-null   object
 3   Harga                1916 non-null   object
 4   Deksripsi            1916 non-null   object
dtypes: object(5)
memory usage: 75.0+ KB


In [ ]:
df_preprocess.duplicated().sum()

41

The dataset contain 41 data duplications

In [ ]:
duplicate_rows = df_preprocess[df_preprocess.duplicated(subset=['ID Produk'], keep=False)]
print(duplicate_rows)

                                 ID Produk Sub Kategori Produk  \
0     066a5e68-7799-4f18-a49a-e2fefc556d1f              Permen   
1     066a5e68-7799-4f18-a49a-e2fefc556d1f              Permen   
118   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion Wanita   
119   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion Wanita   
120   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion Wanita   
...                                    ...                 ...   
1862  17a50ed6-538c-481a-946c-f8e68d3c2069     Fashion Lainnya   
1864  e1f4b3e0-991b-4d60-b019-6efc4141214f      Fashion Wanita   
1865  e1f4b3e0-991b-4d60-b019-6efc4141214f      Fashion Wanita   
1873  cc1f8f30-586e-4d2e-a12f-fd47483a1bb4        Fashion Pria   
1874  cc1f8f30-586e-4d2e-a12f-fd47483a1bb4        Fashion Pria   

                                    Nama Produk   Harga  \
0                                      Supermen   12000   
1                                      Supermen   16000   
118                        Blo

In [ ]:
df_preprocess = df_preprocess.drop_duplicates(subset=['ID Produk'], keep='first')
df_preprocess

,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,<p>Permen susu khas lombok nusa tenggara barat...
2,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,<p>Coklat susu adalah mahakarya gurih yang mem...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...
4,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...
5,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,<p>Lestary Tenun Batik sasambo nlouse Kuning&n...
...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,<p>Mbojo Honey<br></p>
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,<p>Dompet Motif Krealang Sumbawa<br></p>


In [ ]:
df_preprocess.duplicated().sum()

0

### Remove HTML Tag on Deksripsi column

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    text_without_tags = soup.get_text()
    return ' '.join(text_without_tags.split())


df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].apply(remove_html_tags)
df_preprocess

,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...
2,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...
4,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...
5,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M
...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,Mbojo Honey
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa


### Create Concantenated column and Case Foldings Step

Feature yang digunakan dalam column Concatenated adalah gabungan dari column: 

Concantenated = (Nama Produk, Harga dan Deskripsi)

In [ ]:
df_preprocess['Concatenated'] = df_preprocess.apply(lambda row: ' '.join(row[2:]), axis=1)
df_preprocess

,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,Supermen 12000 Permen susu khas lombok nusa te...
2,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,Semeton Choco (Coklat Lombok) 15000 Coklat sus...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,Kopi Lanang 19000 Kopi Lanang Bubuk Ground Lev...
4,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,Kopi Lombok Murni 28000 Kopi denga kemasaran P...
5,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,Lestary Tenun Batik Sasambo Blouse Kuning 6100...
...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,Mbojo Honey,Mbojo Honey 169000 Mbojo Honey
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,Kopi Bubuk 100 Gr 29000 Kopi Bubuk Pesanggerah...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,Kopi Jahe 100 Gr 29000 Kopi Jahe Pesanggerahan...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,Dompet Motif Krealang Sumbawa 455000 Dompet Mo...


In [ ]:
df_preprocess['Concatenated'] = df_preprocess['Concatenated'].str.lower()
df_preprocess


,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,supermen 12000 permen susu khas lombok nusa te...
2,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,semeton choco (coklat lombok) 15000 coklat sus...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi lanang 19000 kopi lanang bubuk ground lev...
4,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi lombok murni 28000 kopi denga kemasaran p...
5,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,lestary tenun batik sasambo blouse kuning 6100...
...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,Mbojo Honey,mbojo honey 169000 mbojo honey
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,kopi bubuk 100 gr 29000 kopi bubuk pesanggerah...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,kopi jahe 100 gr 29000 kopi jahe pesanggerahan...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,dompet motif krealang sumbawa 455000 dompet mo...


### Main Data Preprocessing

In [ ]:
def ganti_isi_column(data_lama,data_baru):
  for i in range(len(data_lama)) :
    data_lama[i] = data_baru[i]
  print(data_lama)

In [ ]:
def preprocessing(document_to_be_scored):
  # Tokenize Dataset
  pisah = {}
  for i in range(len(document_to_be_scored)):
    pisah[i] = document_to_be_scored.values[i].split()

  # Menghilangkan stopword memakai library nltk
  stopword=['dengan'] # custom list stopword

  removed = {}
  hasil = []
  listStopword = stopwords.words('indonesian') # list stopword dari NLTK

  # Tambahkan stopword
  for i in stopword:
    listStopword.append(i)
  print(listStopword,"\n")

  # listStopword = set(stopwords.words('indonesian')) #list stopword
  hitung = 0 #untuk menghitung frekuensi
  for i in pisah.values():
    for j in i:
      if j not in listStopword: #jika setiap kata tidak ada di list stopword
        hasil.append(j)
    removed[hitung] = hasil[:]
    del hasil[:]
    hitung=hitung+1

  # Memakai library sastarawi
  # Memakai library sastrawi untuk melakukan stemming
  factory = StemmerFactory() #membuat objek stemmer
  stemmer = factory.create_stemmer()
  hitung = 0

  hasil=[]
  hasil2 = {}

  for i in removed.values():
    for j in i:
      # Hasil Stemming
      hasil.append(stemmer.stem(j))
    hasil2[hitung] = hasil[:]
    del hasil[:]
    hitung = hitung+1
  hasil =""
  korpus2 =[]
  for i in hasil2.values():
    hasil =""
    for j in i:
      hasil = hasil+ " " +j
    korpus2.append(hasil)
  #Timpa hasil
  ganti_isi_column(document_to_be_scored,korpus2)

8 minutes run

In [ ]:
preprocessing(df_preprocess['Concatenated'])

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [ ]:
df_preprocess.to_csv('clean_preprocessing.csv', index=False)
df_preprocess

,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,supermen 12000 permen susu khas lombok nusa t...
2,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,semeton choco (coklat lombok) 15000 coklat sus...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi lanang 19000 kopi lanang bubuk ground lev...
4,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi lombok murni 28000 kopi denga kemasaran p...
5,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,lestary tenun batik sasambo blouse kuning 6100...
...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,Mbojo Honey,mbojo honey 169000 mbojo honey
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,kopi bubuk 100 gr 29000 kopi bubuk pesanggerah...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,kopi jahe 100 gr 29000 kopi jahe pesanggerahan...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,dompet motif krealang sumbawa 455000 dompet mo...


## Content Base Filtering Algorithm

In [3]:
df_clean = pd.DataFrame(pd.read_csv('clean_preprocessing.csv'))
df_clean

,ID Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,supermen 12000 permen susu khas lombok nusa t...
1,b70def04-da59-4c2e-912a-e5944579831d,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,semeton choco (coklat lombok) 15000 coklat sus...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi lanang 19000 kopi lanang bubuk ground lev...
3,9271605a-009d-4364-8deb-f8d04847e38b,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi lombok murni 28000 kopi denga kemasaran p...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,lestary tenun batik sasambo blouse kuning 6100...
...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Madu,Mbojo Honey,169000,Mbojo Honey,mbojo honey 169000 mbojo honey
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,kopi bubuk 100 gr 29000 kopi bubuk pesanggerah...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,kopi jahe 100 gr 29000 kopi jahe pesanggerahan...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,dompet motif krealang sumbawa 455000 dompet mo...


### Compute TF-IDF Features

In [4]:
def compute_tfidf(documents):
    # Step 1: Tokenization
    def tokenize(document):
        return document.lower().split()

    # Step 2: Create a vocabulary
    vocabulary = set()
    for doc in documents:
        vocabulary.update(tokenize(doc))

    vocabulary = sorted(vocabulary)

    # Step 3: Compute TF (Term Frequency)
    print("\nIterasi pada setiap data:")
    count=1
    tf_matrix = []
    for doc in documents:
        tokens = tokenize(doc)
        tf = [tokens.count(word) for word in vocabulary]
        print("Data Ke-",count,":",tokens)
        tf_matrix.append(tf)
        count+=1


    # Step 4: Compute IDF (Inverse Document Frequency)
    idf = []
    N = len(documents)
    for word in vocabulary:
        doc_count = sum([1 for doc in documents if word in tokenize(doc)])
        idf.append(math.log(N / (1 + doc_count)))


    # Step 5: Compute TF-IDF
    tfidf_matrix = []
    for tf_vector in tf_matrix:
        tfidf = [tf * idf_value for tf, idf_value in zip(tf_vector, idf)]
        tfidf_matrix.append(tfidf)
    
    return vocabulary, tf_matrix, idf, tfidf_matrix

In [ ]:
documents = df_clean['Concatenated']

vocabulary,tf_matrix,idf,tfidf_matrix = compute_tfidf(documents)


Iterasi pada setiap data:
Data Ke- 1 : ['supermen', '12000', 'permen', 'susu', 'khas', 'lombok', 'nusa', 'tenggara', 'barat', 'sehat', 'kandung', 'gula', 'rusak', 'citra', 'susu', 'permen']
Data Ke- 2 : ['semeton', 'choco', '(coklat', 'lombok)', '15000', 'coklat', 'susu', 'adalah', 'mahakarya', 'gurih', 'yang', 'memikat', 'indera', 'kita', 'sejak', 'pertama', 'kali', 'mata', 'kita', 'bertemu', 'dengannya.', 'dengan', 'warna', 'coklat', 'yang', 'menggoda,', 'ia', 'memancarkan', 'aroma', 'manis', 'yang', 'merayu', 'perut', 'dan', 'jiwa.', 'ketika', 'pertama', 'kali', 'gigitan', 'menyentuh', 'bibir,', 'coklat', 'susu', 'itu', 'lembut', 'seperti', 'belaian,', 'mencair', 'dalam', 'mulut', 'dengan', 'kelezatan', 'yang', 'tak', 'terlupakan.', 'rasanya', 'adalah', 'perpaduan', 'sempurna', 'antara', 'manis', 'dan', 'gurih,', 'seperti', 'tarian', 'harmoni', 'rasa', 'yang', 'memanjakan', 'lidah.', 'coklat', 'susu', 'adalah', 'godaan', 'yang', 'sulit', 'untuk', 'ditolak,', 'mengundang', 'kita', '

Save model to csv

In [ ]:
import csv

csv_file_path = 'tfidf_features.csv'

# Save the TF-IDF matrix to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row (vocabulary)
    writer.writerow(vocabulary)

    # Write each TF-IDF vector as a row
    for tfidf_vector in tfidf_matrix:
        writer.writerow(tfidf_vector)

print(f'TF-IDF features saved to {csv_file_path}')

TF-IDF features saved to tfidf_features.csv


### Unique Feature in Sub Kategori Produk

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1858 non-null   object
 1   Sub Kategori Produk  1858 non-null   object
 2   Nama Produk          1858 non-null   object
 3   Harga                1858 non-null   int64 
 4   Deksripsi            1858 non-null   object
 5   Concatenated         1858 non-null   object
dtypes: int64(1), object(5)
memory usage: 87.2+ KB


In [5]:
# Count the occurrences of each unique value in the 'Category' column
value_counts = df_clean['Sub Kategori Produk'].value_counts()

pd.set_option('display.max_rows', None)

print("Value counts in 'Category' column:")
print(value_counts)

Value counts in 'Category' column:
Sub Kategori Produk
Fashion Wanita                        209
Cemilan                               207
Gelang                                129
Kain Tenun Lombok                     116
Kain Songket Lombok                   100
Tas                                    91
Fashion Pria                           86
Kopi                                   82
Kerajinan Gerabah                      52
Kaos                                   46
Minuman Herbal                         44
Kerajinan Ketak                        39
Siap Olah/Siap Saji                    39
Kerajinan Rotan                        28
Dompet                                 28
Perawatan Kulit                        27
Kerajinan Bambu                        27
Madu                                   26
Fashion Lainnya                        23
Kerajinan Lainnya                      22
Coklat                                 21
Makanan Lainnya                        20
Kerajinan Batok dan P

### Euclidean Distance

In [84]:
def euclidean_distance(limit, index,detail_output=False):
    # Baca file CSV yang berisi fitur TF-IDF
    df_tfidf = pd.read_csv('tfidf_features.csv')

    # Pilih kolom yang berisi vektor TF-IDF, misalnya 'tfidf_vector'
    tfidf_matrix = tfidf_matrix = df_tfidf.iloc[::] 

    # Fungsi untuk menghitung Euclidean Distance antara dua vektor
    def euclidean_distance(vector1, vector2):
        return np.linalg.norm(vector1 - vector2)

    # Hitung kemiripan dokumen 1 dengan dokumen lainnya
    document_1 = tfidf_matrix.iloc[index]  # Ganti dengan indeks dokumen yang ingin Anda bandingkan
    label_document_1 = df_clean['Sub Kategori Produk'][index]

    if(detail_output==True):
        print("=====Detail Query Acuan: ======")

        print(label_document_1)
        print(df_clean["ID Produk"][index])
        print(df_clean["Nama Produk"][index])
        print(df_clean["Harga"][index])
        print(df_clean["Deksripsi"][index])
        print('\n\n')


    #digunakan untuk menyimpan hasil perhitungan jarak Euclidean antara dokumen referensi dan dokumen-dokumen lainnya.
    similarities = []

    for i in range(1, len(tfidf_matrix)):
        document_i = tfidf_matrix.iloc[i]
        distance = euclidean_distance(document_1, document_i)
        similarities.append((i, distance))

    # Sort hasil kemiripan berdasarkan jarak (semakin kecil semakin mirip)
    similarities.sort(key=lambda x: x[1])


    count = 0
    # Label dokumen referensi
    label_reference = label_document_1

    # Menginisialisasi list label sebenarnya dan prediksi
    true_labels = []
    predicted_labels = []
    for i, distance in similarities:
        if(detail_output==True):
            print(f"Dokumen {i}: \nEuclidean Distance = {distance}")
            print("ID Document              : ",df_clean['ID Produk'][i])
            print("Sub-Kategori             : ",df_clean['Sub Kategori Produk'][i])
            print("Nama Produk              : ",df_clean['Nama Produk'][i])
            print("Concate Description      : ",df_clean['Concatenated'][i])
            print("\n")

        if (count==limit): break
        else: count+=1

        true_label = df_clean['Sub Kategori Produk'][i]
        true_labels.append(label_document_1)

        # print(label_document_1,"====",true_label)
        
        if label_document_1 == true_label:
            # Dokumen referensi memiliki jarak Euclidean 0, itu adalah dirinya sendiri
            predicted_labels.append(label_reference)
        else:
            # Dokumen dengan jarak Euclidean terkecil dianggap sebagai prediksi
            predicted_labels.append(true_label)


    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

    # Generate classification report
    accuracy_scr = accuracy_score(true_labels, predicted_labels)

    if(detail_output==True):
        confusion_mat = confusion_matrix(true_labels,predicted_labels)
        classification_rep = classification_report(true_labels, predicted_labels)

        print("Classification Report:")
        print(confusion_mat)
        print(classification_rep)

    print(label_document_1," ==> ",accuracy_scr)

    return accuracy_scr

### Cosine Similarity

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_limit(limit, index, detail_output=False):
    # Load the TF-IDF feature matrix (assuming it's already computed)
    df_tfidf = pd.read_csv('tfidf_features.csv')

    # Select the column containing the TF-IDF vectors
    tfidf_matrix = df_tfidf.iloc[::]

    # Calculate the cosine similarity between the reference document and other documents
    reference_vector = tfidf_matrix.iloc[index]  # Reference document
    similarities = cosine_similarity([reference_vector], tfidf_matrix)
    label_document_1 = df_clean['Sub Kategori Produk'][index]
    
    if(detail_output==True):
        print("=====Detail Query Acuan: ======")

        print(label_document_1)
        print(df_clean["ID Produk"][index])
        print(df_clean["Nama Produk"][index])
        print(df_clean["Harga"][index])
        print(df_clean["Deksripsi"][index])
        print('\n\n')

    # Sort the similarities
    similar_indices = (-similarities).argsort()[0][1:limit+1]  # Sort in descending order

    # Initialize lists for true labels and predicted labels
    true_labels = []
    predicted_labels = []

    count=0
    for i in similar_indices:  # Excluding the reference document
        if(detail_output==True):
            similar = similarities[0][i]
            print(f"Dokumen {i}: \nEuclidean Distance = {similar}")
            print("ID Document              : ",df_clean['ID Produk'][i])
            print("Sub-Kategori             : ",df_clean['Sub Kategori Produk'][i])
            print("Nama Produk              : ",df_clean['Nama Produk'][i])
            print("Concate Description      : ",df_clean['Concatenated'][i])
            print("\n")

        if (count==limit): break
        else: count+=1

        true_label = df_clean['Sub Kategori Produk'][i]
        true_labels.append(label_document_1)

        # print(label_document_1,"====",true_label)
        
        if label_document_1 == true_label:
            # Dokumen referensi memiliki jarak Euclidean 0, itu adalah dirinya sendiri
            predicted_labels.append(label_document_1)
        else:
            # Dokumen dengan jarak Euclidean terkecil dianggap sebagai prediksi
            predicted_labels.append(true_label)

    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

    # Generate classification report
    accuracy_scr = accuracy_score(true_labels, predicted_labels)
    
    if(detail_output==True):
        confusion_mat = confusion_matrix(true_labels,predicted_labels)
        classification_rep = classification_report(true_labels, predicted_labels)

        print("Classification Report:")
        print(confusion_mat)
        print(classification_rep)

    print(label_document_1," ==> ",accuracy_scr)

    return accuracy_scr

## Testing

### One sample testing

In [85]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama
import random

limit = 10
n_training = 1
acc=0
index_document = 750

for i in range(n_training):
    euclidean = euclidean_distance(limit,index_document,detail_output=True)
    acc+=euclidean

print("Hasil Evaluasi Akhir: ",acc/n_training)

=====Detail Query Acuan: ======
Fashion Pria
1a7cad85-e9b3-45e3-9e69-5b3caca535ef
Baju Kemeja Kombinasi Pringgesella
572000
Baju Kemeja Kombinasi PringgesellaUkuran : Size LCocok digunakan untuk acara formalBeli sekarang juga hanya di NTB Mall karena di NTB Mall #SemuaAdaSemuaBisa #TheBestChoiceofLocalBrand#produklokal #kerajinan #bambu #NTBMall #oleholehlombok #khasLombok #Lombok #NTB #Sumbawa #BeladanBeliProdukLokal #BanggaBuatanIndonesia #kaintenun #songket #makanankhas #jalanjalan #tempatwisata #marketplaceNTB



Dokumen 750: 
Euclidean Distance = 0.0
ID Document              :  1a7cad85-e9b3-45e3-9e69-5b3caca535ef
Sub-Kategori             :  Fashion Pria
Nama Produk              :  Baju Kemeja Kombinasi Pringgesella
Concate Description      :  baju kemeja kombinasi pringgesella 572000 baju kemeja kombinasi pringgesellaukuran : size lcocok digunakan untuk acara formalbeli sekarang juga hanya di ntb mall karena di ntb mall #semuaadasemuabisa #thebestchoiceoflocalbrand#produklokal #k

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [83]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama
import random

limit = 10
n_training = 1
acc=0
index_document = 750

for i in range(n_training):
    euclidean = cosine_similarity_limit(limit,index_document,detail_output=True)
    acc+=euclidean

print(acc/n_training)

=====Detail Query Acuan: ======
Fashion Pria
1a7cad85-e9b3-45e3-9e69-5b3caca535ef
Baju Kemeja Kombinasi Pringgesella
572000
Baju Kemeja Kombinasi PringgesellaUkuran : Size LCocok digunakan untuk acara formalBeli sekarang juga hanya di NTB Mall karena di NTB Mall #SemuaAdaSemuaBisa #TheBestChoiceofLocalBrand#produklokal #kerajinan #bambu #NTBMall #oleholehlombok #khasLombok #Lombok #NTB #Sumbawa #BeladanBeliProdukLokal #BanggaBuatanIndonesia #kaintenun #songket #makanankhas #jalanjalan #tempatwisata #marketplaceNTB



Dokumen 687: 
Euclidean Distance = 0.4567368469040647
ID Document              :  96fd2cdb-e56c-435e-b77d-550f1788a942
Sub-Kategori             :  Fashion Wanita
Nama Produk              :  Baju Tunik
Concate Description      :  baju tunik 572000 baju tuniksize : all size fit to xlcocok digunakan saat acara formalbeli sekarang juga hanya di ntb mall karena di ntb mall #semuaadasemuabisa #thebestchoiceoflocalbrand#produklokal #kerajinan #bambu #ntbmall #oleholehlombok #khas

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Multiple Testing

In [90]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama
import random

limit = 15
n_training = 1500
acc=0

for i in range(n_training):
    index_document = random.randint(1, 1800)
    euclidean = euclidean_distance(limit,index_document,detail_output=False)
    acc+=euclidean

print("Akurasi rata-rata: ", acc/n_training)

Mainan Kunci  ==>  0.3333333333333333
Cemilan  ==>  0.6666666666666666
Perawatan Kulit  ==>  0.13333333333333333
Kain Tenun Lombok  ==>  0.5333333333333333
Fashion Wanita  ==>  0.2
Fashion Wanita  ==>  0.4666666666666667
Cemilan  ==>  0.6
Kerajinan Lainnya  ==>  0.06666666666666667
Fashion Pria  ==>  0.26666666666666666
Tas  ==>  0.4
Perawatan Kulit  ==>  0.3333333333333333
Cemilan  ==>  0.6
Gelang  ==>  0.4666666666666667
Perawatan Kulit  ==>  0.06666666666666667
Sembako  ==>  0.13333333333333333
Fashion Wanita  ==>  0.8
Kerajinan Gerabah  ==>  0.4
Fashion Wanita  ==>  0.4
Cemilan  ==>  0.5333333333333333
Kain Tenun Lombok  ==>  0.9333333333333333
Kain Tenun Lombok  ==>  0.8
Cemilan  ==>  0.9333333333333333
anting  ==>  0.6
Fashion Pria  ==>  0.7333333333333333
Kain Tenun Lombok  ==>  0.4
Cemilan  ==>  0.5333333333333333
Olahan Ikan  ==>  0.06666666666666667
Fashion Lainnya  ==>  0.3333333333333333
Fashion Wanita  ==>  0.4
Fashion Wanita  ==>  0.26666666666666666
Gelang  ==>  0.4
Kera

In [91]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama
import random

limit = 15
n_training = 1500
acc=0

for i in range(n_training):
    index_document = random.randint(1, 1800)
    euclidean = cosine_similarity_limit(limit,index_document,detail_output=False)
    acc+=euclidean

print("Akurasi rata-rata: ", acc/n_training)

Kerajinan Gerabah  ==>  0.5333333333333333
Kain Tenun Lombok  ==>  0.4
Cemilan  ==>  0.06666666666666667
Gamis  ==>  0.13333333333333333
Fashion Wanita  ==>  0.0
Kerajinan Gerabah  ==>  1.0
Gelang  ==>  1.0
Cemilan  ==>  0.13333333333333333
Madu  ==>  1.0
Siap Olah/Siap Saji  ==>  0.8666666666666667
Perawatan Wajah  ==>  0.13333333333333333
Abon  ==>  0.4666666666666667
Kerajinan Gerabah  ==>  0.0
Kopi  ==>  0.8
Cemilan  ==>  0.6666666666666666
Fashion Wanita  ==>  0.7333333333333333
Coklat  ==>  0.5333333333333333
Fashion Wanita  ==>  0.26666666666666666
Kain Songket Lombok  ==>  0.9333333333333333
Kerajinan Bambu  ==>  0.6
Fashion Wanita  ==>  0.4
Bumbu Dapur  ==>  0.13333333333333333
Kerajinan Bambu  ==>  0.06666666666666667
Kaos  ==>  0.6666666666666666
Gelang  ==>  0.6
Fashion Wanita  ==>  0.5333333333333333
Madu  ==>  1.0
Teh  ==>  0.2
Fashion Pria  ==>  0.9333333333333333
Fashion Wanita  ==>  0.4666666666666667
Fashion Wanita  ==>  0.4
Cemilan  ==>  0.6666666666666666
Cemilan  =